In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

datos = pd.read_csv('cancer.csv')
df = pd.DataFrame(datos)
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [83]:
df = df.drop(['id','Unnamed: 32'], axis=1)

In [84]:
## Revisamos si los datos se encuentran desbalanceados
df.groupby('diagnosis').size()

diagnosis
B    357
M    212
dtype: int64

In [85]:
## PASAMOS LO QUE QUEREMOS PREDECIR A DATOS BINARIOS
df = pd.get_dummies(df, columns=['diagnosis'], drop_first=True)
df.head(2)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis_M
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1


In [86]:
## CREAMOS UN DATAFRAME NUEVO CON LAS CARACTERISTICAS STANDARIZADAS
X = df.iloc[:,0:30]
Y = df['diagnosis_M']
## EN ESTE CASO SERA UN DATAFRAME APARTE SIN EL TARGET

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)


## YA ESTANDARIZADOS HACEMOS EL MODELO
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train,X_test,Y_train,Y_test = train_test_split(X_scaler,Y, test_size=0.20)
model = LogisticRegression(solver='lbfgs',max_iter=1000)
model.fit(X_train,Y_train)
Y_predic = model.predict(X_test)


In [87]:
## EVALUAMOS EL MODELO
model.score(X_test,Y_test)

0.956140350877193

In [88]:
## MATRIZ DE CONFUSION
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_predic,Y_test)
print(matriz)

[[73  4]
 [ 1 36]]


In [89]:
## REPORTE DE LAS DEMAS METRICAS
from sklearn.metrics import classification_report
report = classification_report(Y_predic,Y_test)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.95      0.97        77
           1       0.90      0.97      0.94        37

    accuracy                           0.96       114
   macro avg       0.94      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114



In [90]:
## OBSERVAMOS LA CURVA ROC
from sklearn.metrics import roc_auc_score
curvaroc = roc_auc_score(Y_predic,Y_test)
print(curvaroc)

0.9605124605124605
